In [ ]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os


In [ ]:
#import data
data = pd.read_csv(r'new_removed_tables\FINAL.csv')

In [ ]:
#split data
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.25, random_state=3)

Imputation

In [ ]:
from sklearn.impute import KNNImputer

# Create an instance of the KNNImputer with desired parameters
imputer = KNNImputer(n_neighbors=3, weights='distance')

# Perform KNN imputation on the data
imputed_data_train = imputer.fit_transform(train)

# Convert the imputed data array back to a DataFrame
imputed_train = pd.DataFrame(imputed_data_train, columns=train.columns)

# Display the imputed DataFrame
imputed_train

In [ ]:
from sklearn.impute import KNNImputer

# Create an instance of the KNNImputer with desired parameters
imputer = KNNImputer(n_neighbors=3, weights='distance')

# Perform KNN imputation on the data
imputed_data_test = imputer.fit_transform(test)

# Convert the imputed data array back to a DataFrame
imputed_test = pd.DataFrame(imputed_data_test, columns=test.columns)

# Display the imputed DataFrame
imputed_test

In [ ]:
#drop the tardet
X_train = imputed_train.drop('Average Grades')
y_train = imputed_train['Average Grades']

X_test = imputed_test.drop('Average Grades')
y_test =imputed_test['Average Grades']

Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()


# Perform normalization on the selected columns
X_train  = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)


Models

1. Linear Regression

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, make_scorer

loo = LeaveOneOut()

model_lr = LinearRegression()

# Define the scoring metric
scorer = make_scorer(mean_absolute_error)

# Create a GridSearchCV object with LOOCV

# Initialize variables to store the results
mae_scores = []

for train_index, test_index in loo.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Fit the linear regression model on the training data
    model_lr.fit(X_train_cv, y_train_cv)
    
    # Use the fitted model to predict the value of the left-out data point
    y_pred_LR_CV = model_lr.predict(X_test_cv)
    
    # Calculate the mean squared error of the prediction
    mae = mean_absolute_error(y_test_cv, y_pred_LR_CV)
    
    # Append the MSE score to a list
    mae_scores.append(mae)

# Calculate the average MSE over all data points
avg_mae = np.mean(mae_scores)

# Print the average MSE score
print("Average MAE score:", avg_mae)

In [ ]:
# Save model
import pickle

path = r'new_datasets\models\Linear_regression.pkl'
with open(path, 'wb') as f:
    pickle.dump(model_lr, f)

In [ ]:
# predict on unseen data

lr_pred = model_lr.predict(X_test)

#MAE score
mae = mean_absolute_error(y_test, lr_pred)

In [ ]:
# Plot actual Vs. Predicted Grades

# Reshape y_test to match the shape of lr
y_test_reshaped = np.array(y_test).reshape((1, -1))

import matplotlib.pyplot as plt
import seaborn as sns

# Create a scatter plot
plt.scatter(y_test_reshaped, model_lr, color='blue', alpha=0.5)

# Add a diagonal line indicating perfect predictions
plt.plot([min(y_test_reshaped), max(y_test_reshaped)], [min(y_test_reshaped), max(y_test_reshaped)], color='red')

# Set plot labels and title
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual Grades vs. Linear Regression Predicted Grades')

# Display the plot
plt.show()

In [ ]:
# Histplot of The actudal and the predicted grades


sns.histplot(y_test, label='Ground Truth', kde=True, line_kws={'color': 'green'})
sns.histplot(model_lr, label='Linear Regression Predictions', kde=True, line_kws={'color': 'red'})

# Set plot labels and title
plt.xlabel('Grade')
plt.ylabel('Density')
plt.title('Distribution of Ground Truth vs. Linear Regression Predictions')

# Set the y-axis tick locations and labels
plt.xticks([i/2 for i in range(21)])

plt.xlim(4.0, 11)

# Add legend
plt.legend()

# Display the plot
plt.show()


In [ ]:
# R2 score 
from sklearn.metrics import r2_score

# Calculate the R-squared value
r2 = r2_score(y_test, model_lr)

# Print the R-squared value
print("R-squared:", r2)

Support Vector Regression

In [ ]:
from sklearn.svm import SVR
outer_cv = LeaveOneOut()


outer_results = list()

for train_index, test_index in outer_cv.split(X_train):
    #split data
    X_train_cv, X_test_cv = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    inner_cv = KFold()
    model_svr = SVR()
    parameters = {'kernel': ('rbf','poly'), 'C':[1, 1.5, 10],'gamma': [1e-7, 1e-4],
                                                            'epsilon':[0.1,0.2,0.5,0.3]}
    score = make_scorer(mean_absolute_error)
    
    #define search
    search = GridSearchCV(model_svr, parameters, scoring=score, cv=inner_cv)
    
    #result
    result = search.fit(X_train_cv, np.ravel(y_train_cv))

    #get the best performing fit
    best_model_svr = result.best_estimator_

    #evaluate the model on the hold out dataset
    eval = best_model_svr.predict(X_test_cv)

    mae = mean_absolute_error(y_test_cv, eval)

    #store the results

    outer_results.append(mae)

    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Mean Absolute Error: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))


In [ ]:
# Save model

import pickle

path = r'new_datasets\models\SVR_model.pkl'
with open(path, 'wb') as f:
    pickle.dump(best_model_svr, f)

In [ ]:
# predict on test set

svr_pred = best_model_svr.predict(X_test)

#MAE score
mae = mean_absolute_error(y_test, svr_pred)


In [ ]:
#Regression plot of the predictions and the true values

# Reshape y_test to match the shape of svr
y_test_reshaped = np.array(y_test).reshape((1, -1))

# Create a scatter plot
plt.scatter(y_test_reshaped, best_model_svr, color='blue', alpha=0.5)

# Add a diagonal line indicating perfect predictions
plt.plot([min(y_test_reshaped[0]), max(y_test_reshaped[0])], [min(y_test_reshaped[0]), max(y_test_reshaped[0])], color='red')

# Set plot labels and title
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual Grades vs. Support Vector Regression Predicted Grades')

# Display the plot
plt.show()

In [ ]:
# Histplot of the tredicted and true values
sns.histplot(y_test, color='blue', label='Ground Truth', kde=True)
sns.histplot(best_model_svr, color='red', label='Support Vector Regression Predictions', kde=True)

# Set plot labels and title
plt.xlabel('Grade')
plt.ylabel('Density')
plt.title('Distribution of Ground Truth vs. Support Vector Regression Predictions')

# Set the y-axis tick locations and labels
plt.xticks([i/2 for i in range(21)])

plt.xlim(4.0, 9.5)

# Add legend
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Display the plot
plt.show()

In [ ]:

# Calculate the R-squared value
r2 = r2_score(y_test, best_model_svr)

# Print the R-squared value
print("R-squared:", r2)

Decission Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

for train_index, test_index in outer_cv.split(X_train):
    # Split data
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    cv = KFold()
    tree_cv = DecisionTreeRegressor()
    parameters = {
        "splitter": ["best", "random"],
        "max_depth": [3, 7, 11, 12],
        "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "min_weight_fraction_leaf": [0.1, 0.2],
        "max_features": ["auto", "log2", "sqrt", None],
        "max_leaf_nodes": [10, 20, 40, 50, 90]
    }
    score = make_scorer(mean_absolute_error)
    
    # Define search
    search = GridSearchCV(tree_cv, parameters, scoring=score, cv=cv)
    
    # Result
    result = search.fit(X_train_cv, np.ravel(y_train_cv))

    # Get the best performing fit
    best_model_dt= result.best_estimator_

    # Evaluate the model on the holdout dataset
    eval = best_model_dt.predict(X_test_cv)

    mae = mean_absolute_error(y_test_cv, eval)

    # Store the results
    outer_results.append(mae)

    print('> MAE=%.3f, Est=%.3f, Cfg=%s' % (mae, result.best_score_, result.best_params_))

# Summarize the estimated performance of the model
print('Mean Absolute Error: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))


In [ ]:
# predict on test set

dt_pred = best_model_dt.predict(X_test)

#MAE score
mae = mean_absolute_error(y_test, dt_pred)

In [ ]:
# Save model

import pickle

path = r'new_datasets\models\DT_model.pkl'
with open(path, 'wb') as f:
    pickle.dump(best_model_dt, f)

In [ ]:
# Create a scatter plot
plt.scatter(y_test_reshaped, best_model_dt, color='blue', alpha=0.5)

# Add a diagonal line indicating perfect predictions
plt.plot([min(y_test_reshaped), max(y_test_reshaped)], [min(y_test_reshaped), max(y_test_reshaped)], color='red')

# Set plot labels and title
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual Grades vs. Decision Tree Predicted Grades')

# Display the plot
plt.show()

In [ ]:
#Histplot of predicted and true values

sns.histplot(y_test, color='blue', label='Ground Truth', kde=True)
sns.histplot(best_model_dt, color='red', label='Decision Tree Predictions', kde=True)

# Set plot labels and title
plt.xlabel('Grade')
plt.ylabel('Density')
plt.title('Distribution of Ground Truth vs. Decision Tree Predictions')

# Set the y-axis tick locations and labels
plt.xticks([i/2 for i in range(21)])

plt.xlim(4.0, 9.5)

# Add legend
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Display the plot
plt.show()

In [ ]:
#r2
r2 = r2_score(y_test_reshaped, best_model_dt)

# Print the R-squared value
print("R-squared:", r2)

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

outer_results = list()

outer_cv = LeaveOneOut()

for train_index, test_index in outer_cv.split(X_train):
    #split data
    X_train_cv, X_test_cv = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    inner_cv = KFold()
    rf_cv = RandomForestRegressor()

    param_grid = {
    'n_estimators': [100, 200, 300], 
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]}
        
    score = make_scorer(mean_absolute_error)
    
    #define search
    search = GridSearchCV(rf_cv, param_grid, scoring=score, cv=inner_cv)
    
    #result
    result = search.fit(X_train_cv, np.ravel(y_train_cv))

    #get the best performing fit
    best_model_rf = result.best_estimator_

    #evaluate the model on the hold out dataset
    eval = best_model_rf.predict(X_test_cv)

    mae = mean_absolute_error(y_test_cv, eval)

    #store the results

    outer_results.append(mae)

    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Mean Absolute Error: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

In [ ]:
# predict on test set

rf_pred = best_model_rf.predict(X_test)

#MAE score
mae = mean_absolute_error(y_test, rf_pred)

In [ ]:
# Save model

import pickle

path = r'new_datasets\models\RF_model.pkl'
with open(path, 'wb') as f:
    pickle.dump(best_model_rf, f)

In [ ]:
#Regression plot of true and predicted values

# Create a scatter plot
plt.scatter(y_test_reshaped, best_model_rf, color='blue', alpha=0.5)

# Add a diagonal line indicating perfect predictions
plt.plot([min(y_test_reshaped[0]), max(y_test_reshaped[0])], [min(y_test_reshaped[0]), max(y_test_reshaped[0])], color='red')

# Set plot labels and title
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual Grades vs. Random Forest Predicted Grades')

# Display the plot
plt.show()

In [ ]:
#Histplot of true and predicted values 

sns.histplot(y_test, color='blue', label='Ground Truth', kde=True)
sns.histplot(best_model_rf, color='red', label='Random Forest Predictions', kde=True)

# Set plot labels and title
plt.xlabel('Grade')
plt.ylabel('Density')
plt.title('Distribution of Ground Truth vs. Random Forest Predictions')

# Set the y-axis tick locations and labels
plt.xticks([i/2 for i in range(21)])

plt.xlim(4.0, 9.5)

# Add legend
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Display the plot
plt.show()

In [ ]:
# Calculate the R-squared value
r2 = r2_score(y_test, best_model_rf)

# Print the R-squared value
print("R-squared:", r2)

XGBoost

In [ ]:
from xgboost import XGBRegressor

outer_results = list()

outer_cv = LeaveOneOut()

for train_index, test_index in outer_cv.split(X_train):
    #split data
    X_train_cv, X_test_cv = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    inner_cv = KFold()
    xgboost_cv = XGBRegressor()
    param_grid = {
        'learning_rate': [0.1, 0.3],
        'max_depth': [3, 5],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.6, 0.8]
                                        }
    
    score = make_scorer(mean_absolute_error)
    
    #define search
    search = GridSearchCV(xgboost_cv, param_grid, scoring=score, cv=inner_cv)
    
    #result
    result = search.fit(X_train_cv, y_train_cv)

    #get the best performing fit
    best_model_xgb = result.best_estimator_

    #evaluate the model on the hold out dataset
    eval = best_model_xgb.predict(X_test_cv)

    mae = mean_absolute_error(y_test_cv, eval)

    #store the results

    outer_results.append(mae)

    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Mean Absolute Error: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

In [ ]:
# predict on test set

xgb_pred = best_model_xgb.predict(X_test)

#MAE score
mae = mean_absolute_error(y_test, xgb_pred)

In [ ]:
# Save model

import pickle

path = r'new_datasets\models\XGB_model.pkl'
with open(path, 'wb') as f:
    pickle.dump(best_model_xgb, f)

In [ ]:
# regression plot of true and predicted values

# Create a scatter plot
plt.scatter(y_test_reshaped, best_model_xgb, color='blue', alpha=0.5)

# Add a diagonal line indicating perfect predictions
plt.plot([min(y_test_reshaped), max(y_test_reshaped)], [min(y_test_reshaped), max(y_test_reshaped)], color='red')

# Set plot labels and title
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual Grades vs. XGBoost Predicted Grades')

# Display the
plt.show()

In [ ]:
#Histplot of true and predicted values

sns.histplot(y_test, color='blue', label='Ground Truth', kde=True)
sns.histplot(best_model_xgb, color='red', label='Xgboost Predictions', kde=True)

# Set plot labels and title
plt.xlabel('Grade')
plt.ylabel('Density')
plt.title('Distribution of Ground Truth vs. Xgboost Predictions')

# Set the y-axis tick locations and labels
plt.xticks([i/2 for i in range(21)])

plt.xlim(4.0, 9.5)

# Add legend
plt.legend()

# Display the plot
plt.show()

In [ ]:
r2 = r2_score(y_test_reshaped, xgb)

# Print the R-squared value
print("R-squared:", r2)

Predictor's contribution - SHAP

In [ ]:
import shap
import numpy as np

# Initialize the SHAP explainer
explainer = shap.Explainer(best_model_rf)

# Calculate the SHAP values for all instances in the test set
shap_values = explainer(X_test)

# Get the underlying values from the Explanation object
shap_values_values = shap_values.values

# Calculate the mean absolute SHAP values for each feature
mean_abs_shap_values = np.abs(shap_values_values).mean(axis=0)

# Find the indices of the top 20 most influential values
top_20_indices = np.argsort(mean_abs_shap_values)[-20:]

# Get the corresponding feature names or values
top_20_features = X_test.columns[top_20_indices]  
top_20_values = mean_abs_shap_values[top_20_indices]

In [ ]:
# Plot top 20 SHAP values with corrected names

# Define the custom feature names for the y-axis labels
feature_names = ['Ratio Phone Usage February daily window 24-6', 'N days campus visited February', 'Ratio Phone Usage June daily window 0am-6am', 'Ratio Phone Usage January daily window 12-18', 'Ratio Phone Usage May daily window 24-6', 'Stress trend slope March', 'Ratio Phone Usage May daily window 12-18', 'Energy trend slope March', 'Average time studying February', 'Relaxed trend slope June', 'Average time studying February April', 'Ratio Phone Usage January daily window 24-6', 'Overcoming difficulties', 'Stress overall slope', 'Average hours slept February', 'N days campus visited January', 'Concentration trend slope March', 'Ratio Phone Usage February daily window 12-18', 'Concentration trend slope June', 'Total time spent on campus January']

import matplotlib.pyplot as plt

# Plot the top 20 most influential values
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_20_features)), top_20_values, align='center')
plt.yticks(range(len(top_20_features)), feature_names)
plt.xlabel('Mean Absolute SHAP Value')
plt.ylabel('Features')
plt.title('Top 20 Most Influential Values')
plt.show()